In [ ]:
import getpass
import sys
import os
import os.path
home = os.environ.get("HOME")
sys.path.append(f"{home}/bioinformatics/lib")
sys.path.append(f"{home}/bioinformatics/notebooks/analysis/chronic_pain/publishable/src/lib")

import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
from scipy.sparse import issparse
import anndata
from anndata import AnnData
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from collections import defaultdict, OrderedDict
import gzip
import csv
import re
import io
import logging
import warnings
import subprocess
from preprocessing import *

FORMAT = '%(asctime)-15s %(message)s'
logging.basicConfig(format=FORMAT)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 50)
sc.settings.verbosity = 3
sc.logging.print_header()
sc.settings.set_figure_params(dpi=120, color_map="cividis")

In [ ]:
dataset = "GSE154659"
species = "Mouse"

In [ ]:
dirname = os.getcwd()
source = os.path.abspath(os.path.join(dirname, f'../../build/datasets/{dataset}'))
fns = os.listdir(os.path.join(source, 'downloads'))
print(source)
fns

In [ ]:
#r_script_path = os.path.abspath(os.path.join(dirname, f'../lib/convert_rds_to_mtx.R'))
#for fn in fns:
#    if not fn.endswith('.RDS.gz'):
#        continue
#    input_file_path = os.path.join(source, 'downloads', fn)
#    
#    try:
#        subprocess.run(["gunzip", "-k", input_file_path], check=True)
#        print(f"File decompressed successfully")
#    except subprocess.CalledProcessError as e:
#        print(f"Error during decompression: {e}")
#        
#    try:
#        subprocess.run(["Rscript", r_script_path, input_file_path.rstrip('.gz')], check=True)
#        print("R script executed successfully.")
#    except subprocess.CalledProcessError as e:
#        print(f"Error occurred: {e}")

In [ ]:
# curated metadata from GEO series matrix file

meta = """
sample_id,female_C57_Naive_0_rep1,female_C57_Naive_0_rep2,female_C57_Crush_168_rep1,female_C57_Crush_168_rep2,male_C57_Naive_0_rep1,male_C57_Naive_0_rep2,male_C57_Naive_0_rep3,male_C57_Naive_0_rep4,male_C57_Naive_0_rep5,male_C57_SpNT_6_rep1,male_C57_SpNT_6_rep2,male_C57_SpNT_6_rep3,male_C57_SpNT_12_rep1,male_C57_SpNT_12_rep2,male_C57_SpNT_24_rep1,male_C57_SpNT_24_rep2,male_C57_SpNT_36_rep1,male_C57_SpNT_36_rep2,male_C57_SpNT_48_rep1,male_C57_SpNT_48_rep2,male_C57_SpNT_72_rep1,male_C57_SpNT_72_rep2,male_C57_SpNT_72_rep3,male_C57_SpNT_72_rep4,male_C57_SpNT_72_rep5,male_C57_SpNT_168_rep1,male_C57_SpNT_168_rep2,male_C57_SpNT_168_rep3,male_C57_SpNT_168_rep4,male_C57_Crush_6_rep1,male_C57_Crush_6_rep2,male_C57_Crush_24_rep1,male_C57_Crush_24_rep2,male_C57_Crush_72_rep1,male_C57_Crush_72_rep2,male_C57_Crush_168_rep1,male_C57_Crush_168_rep2,male_C57_Crush_336_rep1,male_C57_Crush_336_rep2,male_C57_Crush_672_rep1,male_C57_Crush_672_rep2,male_C57_Crush_1440_rep1,male_C57_Crush_1440_rep2,male_C57_Crush_2160_rep1,male_C57_Crush_2160_rep2,male_C57_ScNT_6_rep1,male_C57_ScNT_12_rep1,male_C57_ScNT_12_rep2,male_C57_ScNT_24_rep1,male_C57_ScNT_72_rep1,male_C57_ScNT_72_rep2,male_C57_ScNT_168_rep1,male_C57_ScNT_168_rep2,male_C57_ScNT_672_rep1,male_C57_ScNT_672_rep2,male_C57_ScNT_1440_rep1,male_C57_ScNT_1440_rep2,male_C57_Paclitaxel_168_rep1,male_C57_Paclitaxel_168_rep2,male_C57_CFA_48_rep1,male_Mrgprd-cre_Naive_0_rep1,male_Mrgprd-cre_Naive_0_rep2,male_Mrgprd-cre_Naive_0_rep3,male_Mrgprd-cre_Crush_168_rep1,male_Mrgprd-cre_Crush_1440_rep1,male_Mrgprd-cre_Crush_1440_rep2,male_Atf3-WT_Naive_0_rep1,male_Atf3-WT_Naive_0_rep2,male_Atf3-WT_Naive_0_rep3,male_Atf3-WT_Crush_36_rep1,male_Atf3-WT_Crush_168_rep1,male_Atf3-WT_Crush_168_rep2,male_Atf3-KO_Naive_0_rep1,male_Atf3-KO_Naive_0_rep2,male_Atf3-KO_Naive_0_rep3,male_Atf3-KO_Crush_36_rep1,male_Atf3-KO_Crush_168_rep1,male_Atf3-KO_Crush_168_rep2
gsm,GSM4676529,GSM4676530,GSM4676531,GSM4676532,GSM4676533,GSM4676534,GSM4676535,GSM4676536,GSM4676537,GSM4676538,GSM4676539,GSM4676540,GSM4676541,GSM4676542,GSM4676543,GSM4676544,GSM4676545,GSM4676546,GSM4676547,GSM4676548,GSM4676549,GSM4676550,GSM4676551,GSM4676552,GSM4676553,GSM4676554,GSM4676555,GSM4676556,GSM4676557,GSM4676558,GSM4676559,GSM4676560,GSM4676561,GSM4676562,GSM4676563,GSM4676564,GSM4676565,GSM4676566,GSM4676567,GSM4676568,GSM4676569,GSM4676570,GSM4676571,GSM4676572,GSM4676573,GSM4676574,GSM4676575,GSM4676576,GSM4676577,GSM4676578,GSM4676579,GSM4676580,GSM4676581,GSM4676582,GSM4676583,GSM4676584,GSM4676585,GSM4676586,GSM4676587,GSM4676588,GSM4676589,GSM4676590,GSM4676591,GSM4676592,GSM4676593,GSM4676594,GSM4676595,GSM4676596,GSM4676597,GSM4676598,GSM4676599,GSM4676600,GSM4676601,GSM4676602,GSM4676603,GSM4676604,GSM4676605,GSM4676606
sample_source,female_C57_Naive_0,female_C57_Naive_0,female_C57_Crush_168,female_C57_Crush_168,male_C57_Naive_0,male_C57_Naive_0,male_C57_Naive_0,male_C57_Naive_0,male_C57_Naive_0,male_C57_SpNT_6,male_C57_SpNT_6,male_C57_SpNT_6,male_C57_SpNT_12,male_C57_SpNT_12,male_C57_SpNT_24,male_C57_SpNT_24,male_C57_SpNT_36,male_C57_SpNT_36,male_C57_SpNT_48,male_C57_SpNT_48,male_C57_SpNT_72,male_C57_SpNT_72,male_C57_SpNT_72,male_C57_SpNT_72,male_C57_SpNT_72,male_C57_SpNT_168,male_C57_SpNT_168,male_C57_SpNT_168,male_C57_SpNT_168,male_C57_Crush_6,male_C57_Crush_6,male_C57_Crush_24,male_C57_Crush_24,male_C57_Crush_72,male_C57_Crush_72,male_C57_Crush_168,male_C57_Crush_168,male_C57_Crush_336,male_C57_Crush_336,male_C57_Crush_672,male_C57_Crush_672,male_C57_Crush_1440,male_C57_Crush_1440,male_C57_Crush_2160,male_C57_Crush_2160,male_C57_ScNT_6,male_C57_ScNT_12,male_C57_ScNT_12,male_C57_ScNT_24,male_C57_ScNT_72,male_C57_ScNT_72,male_C57_ScNT_168,male_C57_ScNT_168,male_C57_ScNT_672,male_C57_ScNT_672,male_C57_ScNT_1440,male_C57_ScNT_1440,male_C57_Paclitaxel_168,male_C57_Paclitaxel_168,male_C57_CFA_48,male_Mrgprd-cre_Naive_0,male_Mrgprd-cre_Naive_0,male_Mrgprd-cre_Naive_0,male_Mrgprd-cre_Crush_168,male_Mrgprd-cre_Crush_1440,male_Mrgprd-cre_Crush_1440,male_Atf3-WT_Naive_0,male_Atf3-WT_Naive_0,male_Atf3-WT_Naive_0,male_Atf3-WT_Crush_36,male_Atf3-WT_Crush_168,male_Atf3-WT_Crush_168,male_Atf3-KO_Naive_0,male_Atf3-KO_Naive_0,male_Atf3-KO_Naive_0,male_Atf3-KO_Crush_36,male_Atf3-KO_Crush_168,male_Atf3-KO_Crush_168
species,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus
strain,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: C57/Bl6,strain: Mrgprd-CreER;Gcamp6f,strain: Mrgprd-CreER;Gcamp6f,strain: Mrgprd-CreER;Gcamp6f,strain: Mrgprd-CreER;Gcamp6f,strain: Mrgprd-CreER;Gcamp6f,strain: Mrgprd-CreER;Gcamp6f,strain: Atf3f/f,strain: Atf3f/f,strain: Atf3f/f,strain: Atf3f/f,strain: Atf3f/f,strain: Atf3f/f,strain: Vglut2-Cre;Atf3f/f,strain: Vglut2-Cre;Atf3f/f,strain: Vglut2-Cre;Atf3f/f,strain: Vglut2-Cre;Atf3f/f,strain: Vglut2-Cre;Atf3f/f,strain: Vglut2-Cre;Atf3f/f
sex,Sex: female,Sex: female,Sex: female,Sex: female,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male,Sex: male
injury,injury: Naive,injury: Naive,injury: Crush,injury: Crush,injury: Naive,injury: Naive,injury: Naive,injury: Naive,injury: Naive,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: SpNT,injury: Crush,injury: Crush,injury: Crush,injury: Crush,injury: Crush,injury: Crush,injury: Crush,injury: Crush,injury: Crush,injury: Crush,injury: Crush,injury: Crush,injury: Crush,injury: Crush,injury: Crush,injury: Crush,injury: ScNT,injury: ScNT,injury: ScNT,injury: ScNT,injury: ScNT,injury: ScNT,injury: ScNT,injury: ScNT,injury: ScNT,injury: ScNT,injury: ScNT,injury: ScNT,injury: Paclitaxel,injury: Paclitaxel,injury: CFA,injury: Naive,injury: Naive,injury: Naive,injury: Crush,injury: Crush,injury: Crush,injury: Naive,injury: Naive,injury: Naive,injury: Crush,injury: Crush,injury: Crush,injury: Naive,injury: Naive,injury: Naive,injury: Crush,injury: Crush,injury: Crush
time_after_injury,time after injury(h): 0,time after injury(h): 0,time after injury(h): 168,time after injury(h): 168,time after injury(h): 0,time after injury(h): 0,time after injury(h): 0,time after injury(h): 0,time after injury(h): 0,time after injury(h): 6,time after injury(h): 6,time after injury(h): 6,time after injury(h): 12,time after injury(h): 12,time after injury(h): 24,time after injury(h): 24,time after injury(h): 36,time after injury(h): 36,time after injury(h): 48,time after injury(h): 48,time after injury(h): 72,time after injury(h): 72,time after injury(h): 72,time after injury(h): 72,time after injury(h): 72,time after injury(h): 168,time after injury(h): 168,time after injury(h): 168,time after injury(h): 168,time after injury(h): 6,time after injury(h): 6,time after injury(h): 24,time after injury(h): 24,time after injury(h): 72,time after injury(h): 72,time after injury(h): 168,time after injury(h): 168,time after injury(h): 336,time after injury(h): 336,time after injury(h): 672,time after injury(h): 672,time after injury(h): 1440,time after injury(h): 1440,time after injury(h): 2160,time after injury(h): 2160,time after injury(h): 6,time after injury(h): 12,time after injury(h): 12,time after injury(h): 24,time after injury(h): 72,time after injury(h): 72,time after injury(h): 168,time after injury(h): 168,time after injury(h): 672,time after injury(h): 672,time after injury(h): 1440,time after injury(h): 1440,time after injury(h): 168,time after injury(h): 168,time after injury(h): 48,time after injury(h): 0,time after injury(h): 0,time after injury(h): 0,time after injury(h): 168,time after injury(h): 1440,time after injury(h): 1440,time after injury(h): 0,time after injury(h): 0,time after injury(h): 0,time after injury(h): 36,time after injury(h): 168,time after injury(h): 168,time after injury(h): 0,time after injury(h): 0,time after injury(h): 0,time after injury(h): 36,time after injury(h): 168,time after injury(h): 168
tissue,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion,tissue/cell type: Dorsal root ganglion
"""

meta_df = pd.read_csv(io.StringIO(meta), header=None)
meta_df = meta_df.T
meta_df.columns = meta_df.iloc[0]
meta_df  = meta_df[1:]
for c in meta_df.columns:
    meta_df[c] = [x.split(": ")[-1] for x in meta_df[c]]

meta_df["time_after_injury"] = [f"{x}_hrs" for x in meta_df["time_after_injury"]]
meta_df

In [ ]:
#build anndata for all samples:

print ("building adata for C57")
file1 = "GSE154659_C57_Raw_counts.mtx"
bc1 = "GSE154659_C57_barcodes.csv"
genes1 = "GSE154659_C57_genes.csv"

adata = sc.read_mtx(os.path.join(source, 'downloads', file1)).T
bc_df = pd.read_csv(os.path.join(source, 'downloads',bc1), header=0, sep='\t', quotechar='"')
gene_df = pd.read_csv(os.path.join(source, 'downloads', genes1), header=0, sep='\t', quotechar='"')

print (adata.shape)
print (len(bc_df), len(gene_df))

adata.obs_names = list(bc_df["x"])
adata.var_names = list(gene_df["x"])

print ("building adata for Atf3_WT_KO")
file2 = "GSE154659_Atf3_WT_KO_Raw_counts.mtx"
bc2 = "GSE154659_Atf3_WT_KO_barcodes.csv"
genes2 = "GSE154659_Atf3_WT_KO_genes.csv"

adata2 = sc.read_mtx(os.path.join(source, 'downloads', file2)).T
bc_df = pd.read_csv(os.path.join(source, 'downloads', bc2), header=0, sep='\t', quotechar='"')
gene_df = pd.read_csv(os.path.join(source, 'downloads', genes2), header=0, sep='\t', quotechar='"')

adata2.obs_names = list(bc_df["x"])
adata2.var_names = list(gene_df["x"])

print (adata2.shape)
print (len(bc_df), len(gene_df))

adata = adata.concatenate(adata2, join="outer", batch_key="concat_batch")
adata

In [ ]:
adata.obs["sample_id"] = ["_".join(x.split("_")[0:5]) for x in adata.obs_names]
adata.obs = adata.obs.reset_index().merge(meta_df, how="left", on=["sample_id"]).set_index('index')
adata.obs

In [ ]:
# author metadata parsed from sample_id
sexs = []
mouses = []
models = []
hours = []
reps = []
subtypes = []
classes = []
bcs = []

for o in adata.obs_names:
    sp = o.split('_')
    sex = sp[0]
    mouse = sp[1]
    model = sp[2]
    hour = sp[3]
    rep = sp[4]
    subtype = sp[5]
    bc = sp[6]

    if subtype == 'p':
        subtype = 'p_cLTMR2'

    if bc == 'cLTMR2':
        bc = sp[7]

    sexs.append(sex)
    mouses.append(mouse)
    models.append(model)
    hours.append(hour)
    reps.append(rep)
    subtypes.append(subtype)
    bcs.append(bc)

    classes.append('neuron' if subtype in ['NF1', 'NF2', 'NF3', 'PEP1', 'PEP2', 'TRPM8', 'p_cLTMR2', 'SST', 'NP', 'cLTMR1',] else 'non-neuron')

#adata.obs['sex'] = sexs # already there
adata.obs['mouse'] = mouses
adata.obs['model'] = models
adata.obs['hour'] = hours
adata.obs['rep'] = reps
adata.obs['subtype'] = subtypes
adata.obs['class'] = classes
adata.obs['bc'] = bcs
adata.obs

In [ ]:
# process dataset
ensure_unique_idx(adata)
enforce_sparsity(adata)
fix_nan_x(adata)
filter_nan_var_names(adata)
del_raw(adata)
calc_n_genes(adata)
# standardize_gene_symbols(adata, species=species)
filter_low_quality_cells(adata, min_genes=500, min_cells=None, min_counts=None, max_counts=None,
                         min_mt_fraction=None, max_mt_fraction=0.1, use_qc_metrics=True)
adata, doublet_counts = find_doublets(adata, batch_key='gsm')
print(doublet_counts)
normalize_total(adata, target_sum=1e4)
adata

In [ ]:
adata.obs.head()

In [ ]:
adata.var

In [ ]:
fname = os.path.join(source, f"{dataset}.h5ad")
print(fname)
adata.write(fname, compression="gzip")